<a href="https://colab.research.google.com/github/FolarinJoaquim/ADHD-Project-Implementation/blob/main/Subreddit_Scrapper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/FolarinJoaquim/ADHD-Project-Implementation.git

Cloning into 'ADHD-Project-Implementation'...
remote: Enumerating objects: 11, done.
remote: Counting objects: 100% (11/11), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 11 (delta 3), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (11/11), 5.07 KiB | 5.07 MiB/s, done.
Resolving deltas: 100% (3/3), done.


In [ ]:
# All our imports
import requests
import json
import csv
import time
from bs4 import BeautifulSoup


In [ ]:

# Function -> Main Scraper to get content from Reddit
def scrape_reddit() -> list[dict]:
    subreddits = [
        "https://www.reddit.com/r/Python",
        "https://www.reddit.com/r/Programming",
        "https://www.reddit.com/r/learnpython",
    ]

    all_data = []
    for url in subreddits:
        headers = {
              'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
        }

        subreddit_name = url.split("/")[-1]
        print(f'Scraping for: {subreddit_name}')

        try:
            response = requests.get(url, headers=headers, timeout=10)
            response.raise_for_status()

            soup = BeautifulSoup(response.content, "html.parser")

            subreddit_data ={
                "subreddit" : subreddit_name,
                "url" : url,
                "title": soup.title.string if soup.title else "No title",
                "scraped_at" : time.strftime("%Y-%m-%d %H:%M:%S")
            }

            topics = []

            for heading in soup.find_all(["h1","h2","h3","h4"]):
                text = heading.get_text(strip=True)

                if text and len(text) > 3:
                    if any(keyword in text.lower() for keyword in ["python","programming","code","develop","coding"]):
                        topics.append({
                            "title":text,
                            "type": "python_topic"
                        })

            discussions = []
            seen_urls = set()

            for link in soup.find_all("a", href=True):
                text = link.get_text(strip=True)
                href = link["href"]

                if text and len(text) > 1 and "/comments/" in href and href not in seen_urls:
                    seen_urls.add(href)

                    discussions.append({
                        "title": text[:100] + "..." if len(text) > 100 else text,
                        "url":href,
                        "type":"discussion"
                    })

            subreddit_data["python_topics"] = topics
            subreddit_data["discussions"] = discussions

            all_data.append(subreddit_data)
            time.sleep(2)

        except Exception as e:
            print(f'ERROR: {e}')

    return all_data




# Function -> Saved scraped data (Both json and csv)
def save_scraped_data(data, filename_json="python_topics.json", filename_csv="python_topics.csv"):
    if not data:
        print(f'No data')
        return

    # JSON
    try:
        with open(filename_json, "w", encoding="utf-8") as file:
            json.dump(data, file, indent=2, ensure_ascii=True)
        print(f'Python Topics saved: {filename_json}')
    except Exception as e:
        print("ERROR:",e)


    # CSV
    try:
        with open(filename_csv, 'w', newline='', encoding='utf-8') as file:
            writer = csv.writer(file)
            writer.writerow(['Subreddit', 'Type', 'Title', 'URL', 'Scraped At'])

            for subreddit_data in data:
                subreddit = subreddit_data['subreddit']
                scraped_at = subreddit_data['scraped_at']

                for topic in subreddit_data.get('python_topics', []):
                    writer.writerow([subreddit, topic['type'], topic['title'], '', scraped_at])

                for discussion in subreddit_data.get('discussions', []):
                    writer.writerow([subreddit, discussion['type'], discussion['title'], discussion['url'], scraped_at])

            print(f'All topics saved to files!')
    except Exception as e:
        print("Error:",e)




# Main -> Execute the code above
def main() -> None:
    data = scrape_reddit()

    if data:
        print(f'Processing the data...')
        total_topics = 0
        total_discussions = 0

        for subreddit_data in data:
            topics_count = len(subreddit_data.get("python_topics",[]))
            discussions_count = len(subreddit_data.get("discussions",[]))

            total_topics += topics_count
            total_discussions += discussions_count

        print(f'\nTotal: {total_topics}: Python Topics, {discussions_count} Discussions')

        save_scraped_data(data)
    else:
        print("There is not data returned!")




if __name__ == "__main__":
    main()